# Time Series

## About the Data

n this notebook, we will be working with 5 data sets:
- (CSV) Facebook's stock price daily throughout 2018 (obtained using the stock_analysis package).
- (CSV) Facebook's OHLC stock data from May 20, 2019 - May 24, 2019 per minute from Nasdaq.com.
- (CSV) melted stock data for Facebook from May 20, 2019 - May 24, 2019 per minute from Nasdaq.com.
-(DB) stock opening prices by the minute for Apple from May 20, 2019 - May 24, 2019 altered to have seconds in the time from Nasdaq.com.
- (DB) stock opening prices by the minute for Facebook from May 20, 2019 - May 24, 2019 from Nasdaq.com

## Setup

In [ ]:
import numpy as np
import pandas as pd

fb = pd.read_csv('/content/fb_2018.csv', index_col='date', parse_dates=True).assign(
trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

## Time-based selection and filtering

Remember, when we have a DatetimeIndex , we can use datetime slicing. We can provide a range of dates. We only get three days back because the stock market is
closed on the weekends:

In [ ]:
fb['2018-10-11':'2018-10-15']

In [ ]:
fb['2018-q1'].equals(fb['2018-01':'2018-03'])

The first() method will give us a specified length of time from the beginning of the time series. Here, we ask for a week. January 1, 2018 was a holiday—meaning the
market was closed. It was also a Monday, so the week here is only four days:

In [ ]:
fb.first('1W')

The last() method will take from the end

In [ ]:
fb.last('1W')

For the next few examples, we need datetimes, so we will read in the stock data per minute file:

In [ ]:
stock_data_per_minute = pd.read_csv(
  '/content/fb_week_of_may_20_per_minute.csv', index_col='date', parse_dates=True,
  date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H-%M')
)
stock_data_per_minute.head()

We can use the Grouper to roll up our data to the daily level along with first and last

In [ ]:
stock_data_per_minute.groupby(pd.Grouper(freq='1D')).agg({
  'open': 'first',
  'high': 'max',
  'low': 'min',
  'close': 'last',
  'volume': 'sum'
})

The at_time() method allows us to pull out all datetimes that match a certain time. Here, we can grab all the rows from the time the stock market opens (9:30 AM):

In [ ]:
stock_data_per_minute.at_time('9:30')

We can use between_time() to grab data for the last two minutes of trading daily

In [ ]:
stock_data_per_minute.between_time('15:59', '16:00')

On average, are more shares traded within the first 30 minutes of trading or in the last 30 minutes? We can combine between_time() with Groupers and filter()
from the aggregation.ipynb notebook to answer this question. For the week in question, more are traded on average around opening time than closing time:

In [ ]:
shares_traded_in_first_30_min = stock_data_per_minute\
  .between_time('9:30', '10:00')\
  .groupby(pd.Grouper(freq='1D'))\
  .filter(lambda x: (x.volume > 0).all())\
  .volume.mean()
shares_traded_in_last_30_min = stock_data_per_minute\
  .between_time('15:30', '16:00')\
  .groupby(pd.Grouper(freq='1D'))\
  .filter(lambda x: (x.volume > 0).all())\
  .volume.mean()

shares_traded_in_first_30_min - shares_traded_in_last_30_min

In cases where time doesn't matter, we can normalize the times to midnight:

In [ ]:
pd.DataFrame(
  dict(before=stock_data_per_minute.index, after=stock_data_per_minute.index.normalize())
).head()

Note that we can also use normalize() on a Series object after accessing the dt attribute:

In [ ]:
stock_data_per_minute.index.to_series().dt.normalize().head()

## Shifting for lagged data

We can use shift() to create some lagged data. By default, the shift will be one period. For example, we can use shift() to create a new column that indicates the
previous day's closing price. From this new column, we can calculate the price change due to after hours trading (after the close one day right up to the open the following
day):

In [ ]:
fb.assign(
  prior_close=lambda x: x.close.shift(),
  after_hours_change_in_price=lambda x: x.open - x.prior_close,
  abs_change=lambda x: x.after_hours_change_in_price.abs()
).nlargest(5, 'abs_change')

The tshift() method will shift the DatetimeIndex rather than the data. However, if the goal is to to add/subtract time we can use pd.Timedelta :

In [ ]:
pd.date_range('2018-01-01', freq='D', periods=5) + pd.Timedelta('9 hours 30 minutes')

When working with stock data, we only have data for the dates the market was open. We can use first_valid_index() to give us the index of the first non-null entry in
our data. For September 2018, this is September 4th:

In [ ]:
fb['2018-09'].first_valid_index()

Conversely, we can use last_valid_index() to get the last entry of non-null data. For September 2018, this is September 28th:

In [ ]:
fb['2018-09'].last_valid_index()

We can use asof() to find the last non-null data before the point we are looking for, if it isn't in the index. From the previous result, we know that the market was not open
on September 30th. It also isn't in the index

In [ ]:
fb.index.asof('2018-09-30')

If we ask for it, we will get the data from the index we got from fb['2018-09'].last_valid_index() , which was September 28th:

In [ ]:
fb.asof('2018-09-30')

## Differenced data

Using the diff() method is a quick way to calculate the difference between the data and a lagged version of it. By default, it will yield the result of data -
data.shift()

In [ ]:
(
  fb.drop(columns='trading_volume')
  - fb.drop(columns='trading_volume').shift()
).equals(
  fb.drop(columns='trading_volume').diff()
)

We can use this to see how Facebook stock changed day-over-day

In [ ]:
fb.drop(columns='trading_volume').diff().head()

We can specify the number of periods, can be any positive or negative integer

In [ ]:
fb.drop(columns='trading_volume').diff(-3).head()

## Resampling

Sometimes the data is at a granularity that isn't conducive to our analysis. Consider the case where we have data per minute for the full year of 2018. Let's see what happens
if we try to plot this.

Plotting will be covered in the next module, so don't worry too much about the code.

First, we import matplotlib for plotting

In [ ]:
import matplotlib.pyplot as plt

Then we will look at the plot at the minute level and at the daily aggregated level (summed):

In [ ]:
np.random.seed(0)
index = pd.date_range('2018-01-01', freq='T', periods=365*24*60)
raw = pd.DataFrame(
  np.random.uniform(0, 10, size=index.shape[0]), index=index
)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
raw.plot(legend=False, ax=axes[0], title='raw data')
raw.resample('1D').sum().plot(legend=False, ax=axes[1], title='daily totals')
for ax in axes:
  ax.set_xlabel('date')
  ax.set_ylabel('events')

plt.suptitle('Raw versus Resampled Data')
plt.show()

The plot on the left has so much data we can't see anything. However, when we aggregate to the daily totals, we see the data. We can alter the granularity of the data we are
working with using resampling. Recall our minute-by-minute stock data:

In [ ]:
stock_data_per_minute.head()

We can resample this to get to a daily frequency:

In [ ]:
stock_data_per_minute.resample('1D').agg({
  'open': 'first',
  'high': 'max',
  'low': 'min',
  'close': 'last',
  'volume': 'sum'
})

We can downsample to quarterly data:

In [ ]:
fb.resample('Q').mean()

We can also use apply() . Here, we show the quarterly change from start to end:

In [ ]:
fb.drop(columns='trading_volume').resample('Q').apply(
  lambda x: x.last('1D').values - x.first('1D').values
)

Consider the following melted stock data by the minute. We don't see the OHLC data directly

In [ ]:
melted_stock_data = pd.read_csv('/content/melted_stock_data.csv', index_col='date', parse_dates=True)
melted_stock_data.head()

We can use the ohlc() method after resampling to recover the OHLC columns:

In [ ]:
melted_stock_data.resample('1D').ohlc()['price']

Alternatively, we can upsample to increase the granularity. Note this will introduce NaN values:

In [ ]:
fb.resample('6H').asfreq().head()

There are many ways to handle these NaN values. We can forward-fill with pad()

In [ ]:
fb.resample('6H').pad().head()

We can specify a specific value or a method with fillna() :

In [ ]:
fb.resample('6H').fillna('nearest').head()

We can use asfreq() and assign() to specify the action per column

In [ ]:
fb.resample('6H').asfreq().assign(
  volume=lambda x: x.volume.fillna(0), # put 0 when market is closed
  close=lambda x: x.close.fillna(method='ffill'), # carry forward
  # take the closing price if these aren't available
  open=lambda x: np.where(x.open.isnull(), x.close, x.open),
  high=lambda x: np.where(x.high.isnull(), x.close, x.high),
  low=lambda x: np.where(x.low.isnull(), x.close, x.low)
).head()

## Merging

We saw merging examples the querying_and_merging notebook. However, they all matched based on keys. With time series, it is possible that they are so granular that we
never have the same time for multiple entries. Let's work with some stock data at different granularities:

In [ ]:
import sqlite3

with sqlite3.connect('/content/stocks.db') as connection:
  fb_prices = pd.read_sql(
    'SELECT * FROM fb_prices', connection,
    index_col='date', parse_dates=['date']
  )
  aapl_prices = pd.read_sql(
    'SELECT * FROM aapl_prices', connection,
    index_col='date', parse_dates=['date']
  )

The Facebook prices are at the minute granularity

In [ ]:
fb_prices.index.second.unique()

However, the Apple prices have information for the second

In [ ]:
aapl_prices.index.second.unique()

We can perform an asof merge to try to line these up the best we can. We specify how to handle the mismatch with the direction and tolerance parameters. We will
fill in with the direction of nearest and a tolerance of 30 seconds. This will place the Apple data with the minute that it is closest to, so 93152 will go with 932
and 93707 will go with 937. Since the times are on the index, we pass left_index and right_index , as we did with merges earlier this chapter

In [ ]:
pd.merge_asof(
  fb_prices, aapl_prices,
  left_index=True, right_index=True, # datetimes are in the index
  # merge with nearest minute
  direction='nearest', tolerance=pd.Timedelta(30, unit='s')
).head()

If we don't want to lose the seconds information with the Apple data, we can use pd.merge_ordered() instead, which will interleave the two. Note this is an outer join by
default ( how parameter). The only catch here is that we need to reset the index in order to join on it:

In [ ]:
pd.merge_ordered(
  fb_prices.reset_index(), aapl_prices.reset_index()
).set_index('date').head()

We can pass a fill_method to handle NaN values

In [ ]:
pd.merge_ordered(
  fb_prices.reset_index(), aapl_prices.reset_index(),
  fill_method='ffill'
).set_index('date').head()